In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

In [1]:
import requests

url = "https://www.leyecoiv.com/rates/breakdown-of-generation-charges/"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the webpage
    html_content = response.text

    # Save the content to a file
    with open("leyeco_iv_webpage.html", "w", encoding="utf-8") as file:
        file.write(html_content)
    print("Webpage source code downloaded and saved as leyeco_iv_webpage.html")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Webpage source code downloaded and saved as leyeco_iv_webpage.html


In [8]:
import re

# Read the HTML content from the file
with open('leyeco_iv_webpage.html', 'r') as file:
    html_content = file.read()

# Find all href links that match the specified format
pattern = r'href=["\'](https://www\.leyecoiv\.com/\d{4}/\d{2}/\d{2}/[^"\']*?\d{4}/)["\']'
links = re.findall(pattern, html_content)

# Display the extracted links
for link in links:
    print(link)

https://www.leyecoiv.com/2024/01/25/january-2024/
https://www.leyecoiv.com/2024/01/25/january-2024/
https://www.leyecoiv.com/2024/02/23/february-2024/
https://www.leyecoiv.com/2024/02/23/february-2024/
https://www.leyecoiv.com/2024/03/25/march-2024/
https://www.leyecoiv.com/2024/03/25/march-2024/
https://www.leyecoiv.com/2024/04/24/april-2024/
https://www.leyecoiv.com/2024/04/24/april-2024/
https://www.leyecoiv.com/2024/05/21/may-2024/
https://www.leyecoiv.com/2024/05/21/may-2024/
https://www.leyecoiv.com/2024/06/24/june-2024/
https://www.leyecoiv.com/2024/06/24/june-2024/
https://www.leyecoiv.com/2024/07/24/july-2024/
https://www.leyecoiv.com/2024/07/24/july-2024/
https://www.leyecoiv.com/2023/01/26/january-2023/
https://www.leyecoiv.com/2023/01/26/january-2023/
https://www.leyecoiv.com/2023/02/15/february-2023/
https://www.leyecoiv.com/2023/02/15/february-2023/
https://www.leyecoiv.com/2023/03/23/march-2023/
https://www.leyecoiv.com/2023/03/23/march-2023/
https://www.leyecoiv.com/202